In [1]:
import nltk
from ebooklib import epub 
from  comFunc import *
import os 
import glob
from multiprocessing import Pool
import time
import re, string
import functools

In [72]:
d_dir = '/home/some/txtsn/'
cwd = !pwd
cwd = cwd[0]
os.chdir(d_dir)
txtfiles = glob.glob('*.txt')
os.chdir(cwd)
print (len(txtfiles))

keywords = list(s.replace('\n','') for s in open('data/add_keywords.txt').readlines()) + list(s.replace('\n','') for s in open('data/keywords.txt').readlines())


14732


In [73]:
def writeTag(fname,tag):
    key='http://purl.org/dc/elements/1.1/'
  
    #try:
    if os.path.exists(fname):
        try:
            #print(fname)
            book=epub.read_epub(fname)
            os.remove(fname)
        except :
            print('error readinng',fname)
            os.remove(fname)
            return
        if 'subject' in book.metadata[key].keys():
            tgs=[b[0].lower().strip() for b in book.metadata[key]['subject'] if b[0] != None]
        else:
            tgs=[]
        ch_cnt=0
        for t in tag:
            t=t.lower().strip()
            if t not in tgs:
                tt=(t,{})
                if 'subject' in book.metadata[key].keys():
                    book.metadata[key]['subject'].append(tt)
                    ch_cnt+=1
                else:
                    book.metadata[key]['subject']=list()
                    book.metadata[key]['subject'].append(tt)
                    ch_cnt+=1
        if ch_cnt>0:
            tmp="/tmp/writeTag"+str(time.time())+'.epub'
            try:
                epub.write_epub(tmp,book)
                
            except:
                print('error writing',fname)
                return
            if os.path.exists(tmp):
                
                !mv "$tmp" "$fname"
    #except :
     #   print('bad file  ',fname)
        #os.remove(fname)
def generate(f):
    tp = '/home/some/txtsn/'
    ep = '/media/some/sophie-hhd/books/thrill/'
    ft = tp + f
    fe = ep + f[:-3] + "epub"
    #try:
    fl= open(ft,'r')
    txt=fl.read()
    fl.close()
    txt=''.join(''.join(tt for tt in t if tt in string.printable) for t in txt).lower()
    txt=txt.replace('\n',' ')
    txt=re.sub(r'[^a-zA-Z0-9 -]*','',txt)
    #hist=nltk.FreqDist(txt.split())
    #res=[[l,hist[l.strip()]] for l in keywords if l.strip() in hist.keys() ]
    res=[[l,txt.count(l)] for l in keywords]
    tag=[r[0].strip() for r in res if r[1]>2]
    #tag=', '.join([r[0].strip() for r in res if r[1]>0])
    #print (txt)
    if len(tag)>0:
        #!ebook-meta "$fe" --tag "$tag" > /dev/null
        
        writeTag(fe,tag)
        return [f[:-4],{r[0]:str(r[1]) for r in res},tag]#,dict(hist)
    #except:
    #print('invalid   ',f)

In [39]:
tags={}#load_obj('data/tagsIm')
keys=[]#load_obj('data/tagskeysIm')


In [75]:
#tags=load_obj('data/tagsIm')
#keys=load_obj('data/tagskeysIm')
t0 = time.time()
p = Pool(10)
books=p.map(generate, list(filter(lambda x: x[:-4] not in keys, txtfiles)))
#p.join()
p.terminate()
p.close()


for book in books:
    try:
        if book[0] not in keys:
            tags[book[0]]=[book[1],book[2]]
    except:
        print(type(book))

save_obj(tags, 'data/tagsIm')
keys=keys+list(tags.keys())
save_obj(keys,'data/tagskeysIm')
t1=time.time()
total = t1-t0
print('time: ',total)

error readinng /media/some/sophie-hhd/books/thrill/Arms and the men.epub
error writing /media/some/sophie-hhd/books/thrill/Edgar Huntly - Charles Brockden Brown.epub
error writing /media/some/sophie-hhd/books/thrill/Orion - Ben Bova.epub
error readinng /media/some/sophie-hhd/books/thrill/The Blood Knight - Greg Keyes.epub
error readinng /media/some/sophie-hhd/books/thrill/Fury - Salman Rushdie.epub
error writing /media/some/sophie-hhd/books/thrill/'Salem's Lot - Stephen King.epub
error writing /media/some/sophie-hhd/books/thrill/Orion in the Dying Time - Ben Bova.epub


/usr/local/lib/python3.5/dist-packages/ebooklib/epub.py:848: UserWarning: Duplicate name: 'EPUB/Images/cover0001.jpg'
  self.out.writestr('%s/%s' % (self.book.FOLDER_NAME, item.file_name), item.get_content())


error writing /media/some/sophie-hhd/books/thrill/Walter Raleigh - Robert Louis Stevenson.epub
error writing /media/some/sophie-hhd/books/thrill/Dead calm - Charles Williams.epub
error writing /media/some/sophie-hhd/books/thrill/The Heads of Cerberus - Francis Stevens.epub
error writing /media/some/sophie-hhd/books/thrill/Eclipse One_ new science fictio - Jonathan Strahan.epub
error writing /media/some/sophie-hhd/books/thrill/Far Off Things - Arthur Machen.epub
error readinng /media/some/sophie-hhd/books/thrill/Picture and Text - Henry James.epub
error readinng /media/some/sophie-hhd/books/thrill/The Phantom Rickshaw - Rudyard Kipling.epub
error writing /media/some/sophie-hhd/books/thrill/The Body-Snatcher - Robert Louis Stevenson.epub
error writing /media/some/sophie-hhd/books/thrill/The gypsy morph - Terry Brooks.epub
error readinng /media/some/sophie-hhd/books/thrill/Frederick Forsyth - The Day of the Jackal.epub
error writing /media/some/sophie-hhd/books/thrill/Memoirs of Carwin th

In [54]:
p.terminate()
p.close()


In [76]:
len(list(filter(lambda x: x[:-4] not in keys, txtfiles))),len(txtfiles)

(207, 14732)

In [67]:
#list(filter(lambda x: x[:-4] not in keys, txtfiles))

In [77]:
cnt=1
kk=[f[:-4] for f in txtfiles]
for t in kk:
    if t in  tags.keys():
        print (cnt,"-",t,": ",' '.join(tags[t][1]))
        cnt+=1
    #print (kk)

1 - Reality and Dreams - Muriel Spark :  actor famous love star wedding millionaire passion car phone millionaire blood pain illionaire
2 - Desolation Island - Patrick O'Brian :  famous love virgin prince wealthy star passion kiss car stalk protect blood bleed pain abuse
3 - Powers - Ursula K. le Guin :  famous love marry virgin wealthy star boss passion kiss car protect blood pain rape boss torture
4 - The Metamorphoses of Ovid Vol. I, Books I-VII - Ovid :  actor famous love marry virgin adore prince star passion kiss car protect blood pain abuse rape
5 - Binary - Michael Crichton :  star car phone protect blood pain
6 - The Hunger - Whitley Strieber :  love star passion kiss romantic car phone protect blood pain rape kidnap
7 - The Age of Miracles - Karen Thompson Walker :  hollywood love romance star wedding kiss car phone protect blood bleed pain
8 - The House of Wittgenstein_ A Family at W - Alexander Waugh :  famous love marry virgin adore prince star wedding diamond billionaire 